In [ ]:
import os
import requests 
import xmltodict
import json
import random
from datetime import datetime, timedelta
import threading
import time

rates = ["EUR", "GBP", "USD", "DZD", "AUD", "BWP", "BND", "CAD", "CLP", "CNY", "COP",
         "CZK", "DKK", "HUF", "ISK", "INR", "IDR", "ILS", "KZT", "KRW", "KWD", "LYD",
         "MYR", "MUR", "NPR", "NZD", "NOK", "OMR", "PKR", "PLN", "QAR", "RUB", "SAR",
         "SGD", "ZAR", "LKR", "SEK", "CHF", "THB", "TTD"]

rates_for_base = [r for r in rates if r not in ["USD", "EUR", "GBP"]]
base = random.choice(rates_for_base)


start = datetime(2011, 5, 4)
end = datetime.today()
dates = []
while start <= end:
    dates.append(start.strftime("%Y-%m-%d"))
    start += timedelta(days=1)

folder = "data_" + base
os.makedirs(folder, exist_ok=True)

def download(date):
    path = f"{folder}/{date}.json"
    if os.path.exists(path):
        return
    url = f"https://www.floatrates.com/historical-exchange-rates.html?operation=rates&pb_id=1775&page=historical&currency_date={date}&base_currency_code={base}&format_type=xml"
    try:
        r = requests.get(url, timeout=10)
        d = xmltodict.parse(r.text)
        with open(path, "w") as f:
            json.dump(d, f)
        print("Saved", date)
    except:
        print("Failed", date)

def run_all():
    i = 0
    while i < len(dates):
        group = []
        for j in range(10):
            if i + j < len(dates):
                t = threading.Thread(target=download, args=(dates[i + j],))
                t.start()
                group.append(t)
        for t in group:
            t.join()
        i += 10

print("Starting", base)
run_all()
print("Done")